In [40]:
#import here
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import multiprocessing as mp
from lxml import etree

In [7]:
print ("Count CPU : ",mp.cpu_count())

Count CPU :  4


In [16]:
df = pd.read_excel('ListofURLsv2.xlsx', sheet_name=0)   
mylist = df['Listof1000URLs'].tolist()

In [27]:
result = len(mylist)
mylist[0]

'https://www.glbtnearme.org/GLBTNearMeResults.php?zip=36925&miles=500&category=Youth'

In [28]:
ua = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36 Edg/92.0.902.62"}
response = requests.get(mylist[0]) 
status = response.status_code
status
 

200

In [29]:
soup = BeautifulSoup(response.content, 'html.parser')

In [31]:
results = soup.find('div', {'class':'categoryresults'}).find_all('p')

In [135]:
#Tanpa HTML Tag
dom = etree.HTML(str(results[0]))
name = dom.xpath('//p[1]/text()[1]')[0] 
x = name.split("\xa0 \xa0\xa0 \xa0\xa0 \xa0")
x[0]

'Mississippi School for Mathematics and Science'

In [83]:
#Dengan HTML Tag
results[1]
name = results[1].find_all('a')
print(name[0].text)

Brown, Jane


In [195]:
#Cek Web & Email
results[1]
rsweb = results[0].find_all('a',href=True)  
web=[]
mail=[]
for urlw in rsweb:
    url = urlw['href']
    if url.startswith('https') or url.startswith('http'):
        exists = url in web
        if exists == False:
            web.append(url)
    elif url.startswith('mailto'):
        urlr = url.replace('mailto:', '')
        exists = urlr in mail
        if exists == False:
            mail.append(urlr)
webj = ",". join(web)
mailj = ",". join(mail)
print(webj)
print(mailj)


woodsm21@themsms.org


In [196]:
 #Gabung
source_url = []
name = []
website = []
email = []
for ress in results:
    source_url.append(mylist[0])
    #Search Name
    dom = etree.HTML(str(ress))
    na = dom.xpath('//p[1]/text()[1]')[0] 
    namesr = na.split("\xa0 \xa0\xa0 \xa0\xa0 \xa0") 
    if namesr[0] != '': 
        name.append(namesr[0])
    else:
        src1 = ress.find_all('a',href=True)
        if not src1:
            name.append('-')
        else:
            name.append(src1[0].text)  
    
    #Seach WEB & MAIL
    src2 = ress.find_all('a',href=True)
    if not src2:
        website.append('-')
        email.append('-')
    else:  
        web=[]
        mail=[]
        for urlw in src2:
            url = urlw['href']
            if url.startswith('https') or url.startswith('http'):
                exists = url in web
                if exists == False:
                    web.append(url)
            elif url.startswith('mailto'):
                urlr = url.replace('mailto:', '')
                exists = urlr in mail
                if exists == False:
                    mail.append(urlr) 
        if not web:
            website.append('-')
        else:
            webj = ",". join(web)
            website.append(webj)
            
        if not mail:
            email.append('-') 
        else:
            mailj = ",". join(mail)
            email.append(mailj) 

In [197]:
#Source URL	Name	Web	Email
datasave = pd.DataFrame({'Source URL': source_url, 'Name':name, 'Web':website, 'Email':email})
datasave

,Source URL,Name,Web,Email
0,https://www.glbtnearme.org/GLBTNearMeResults.p...,Mississippi School for Mathematics and Science,-,woodsm21@themsms.org
1,https://www.glbtnearme.org/GLBTNearMeResults.p...,"Brown, Jane",http://www.adultandchilddevelopment.com,ljsmith@adultandchilddevelopment.com
2,https://www.glbtnearme.org/GLBTNearMeResults.p...,Mississippi Safe Schools Coalition,"http://www.mssafeschools.org/,http://twitter.c...",info@mssafeschools.org
3,https://www.glbtnearme.org/GLBTNearMeResults.p...,Steel City Spectrum,http://www.facebook.com/steelcityspectrum,keller@mcac-bao.org
4,https://www.glbtnearme.org/GLBTNearMeResults.p...,"Willis, Samantha","http://avail.services,http://openpathcollectiv...",swillis@avail.services
...,...,...,...,...
178,https://www.glbtnearme.org/GLBTNearMeResults.p...,Fuse,"http://www.dfwfuse.com,http://www.facebook.com...",fusebox@myresourcecenter.org
179,https://www.glbtnearme.org/GLBTNearMeResults.p...,"Promise House, Inc.","http://www.promisehouse.org,http://www.faceboo...",-
180,https://www.glbtnearme.org/GLBTNearMeResults.p...,Rock Bridge High School Gay Straight Alliance,-,gkirchhofer@cpsk12.org
181,https://www.glbtnearme.org/GLBTNearMeResults.p...,DFWTKF,"http://dfwtkf.com,http://www.facebook.com/pg/D...",info@dfwtkf.com


In [198]:
datasave.to_excel('data20220605.xlsx', index=False)